In [48]:
import sys
sys.path.append("../")  # 상위 디렉토리 추가
from asopipe.main import ASOdesign
import multiprocessing as mp
import time
# ────────────────────────────
# ──────────────────────────────────────────────────────────────
# 3) 실행 예시  ────────────────────────────────────────────────
if __name__ == "__main__":
    mp.set_start_method("spawn", force=True)   # macOS/Linux 안전

    t0   = time.time()
    transid = "NM_133379"#TTN
    #transid="NM_002415" #MIF
    aso  = ASOdesign(transid=transid,
                     query_assembly=["mm39","mm10"],
                     ref_assembly="hg38",
                     tile_length=18)

    result = aso.process_main(chunk_division=5, max_workers=4, wobble=2, to_df=False)
    print("Elapsed:", time.time() - t0, "sec")

[ASOdesign] transid=NM_133379
#tiles=62998, assemblies=['mm39', 'mm10']
Elapsed: 0.8598206043243408 sec
Elapsed: 1.8540451526641846 sec
Elapsed: 2.6885743141174316 sec
Elapsed: 3.3390073776245117 sec
Elapsed: 4.028864622116089 sec
Elapsed: 4.02903938293457 sec
[mm39]
[mm10]
finished. first 3 results: [{'hg38': 'TTTATATTTGTCGAGTCT', 'mm39': 'TTCATATTTATTATGTCT'}]
finished. first 3 results: [5]
Elapsed: 107.24160194396973 sec


In [50]:
import pandas as pd
pd.DataFrame(result)['wobble_mm39'].value_counts()

False                                                                        52574
{'GU_humanC': [], 'GU_otherC': [], 'I_humanC': [], 'I_otherwise': []}         3598
{'GU_humanC': [], 'GU_otherC': [0], 'I_humanC': [], 'I_otherwise': [0]}         55
{'GU_humanC': [], 'GU_otherC': [15], 'I_humanC': [], 'I_otherwise': [15]}       51
{'GU_humanC': [0], 'GU_otherC': [], 'I_humanC': [], 'I_otherwise': []}          51
                                                                             ...  
{'GU_humanC': [], 'GU_otherC': [3], 'I_humanC': [], 'I_otherwise': [15]}         1
{'GU_humanC': [], 'GU_otherC': [4], 'I_humanC': [], 'I_otherwise': [16]}         1
{'GU_humanC': [], 'GU_otherC': [5], 'I_humanC': [], 'I_otherwise': [17]}         1
{'GU_humanC': [], 'GU_otherC': [], 'I_humanC': [12], 'I_otherwise': [6]}         1
{'GU_humanC': [], 'GU_otherC': [], 'I_humanC': [], 'I_otherwise': [8, 9]}        1
Name: wobble_mm39, Length: 2261, dtype: int64

In [69]:
import traceback

def _getWingCoord(result, middle_size=10, gapmer_coord=''):
    try:
        coordL = [] 
        for sequence, length in zip(result["Sequence"], result["Length"]):
            if length < middle_size:
                raise ValueError(f"Sequence Length is {length}")
            wing_coord = None
            gap_seq = ''
            if gapmer_coord!='':
                wing_coord = tuple(map(int, gapmer_coord.split('_')))
                gap_seq = sequence[wing_coord[0]:-wing_coord[2]]
                coordL.append( [(wing_coord, gap_seq), None] )
            else:
                # sequence length가 2의 배수 
                if length % 2 == 0:
                    # if length == 20
                    # (20-10) //2 = 5 -> (5, 10, 5) seq[5:-5] result: (5,10,5),가운데 10개 seq 
                    wing_size = (length-middle_size)//2
                    wing_coord= (wing_size,middle_size,wing_size)
                    gap_seq   = sequence[wing_size: -wing_size]
                    coordL.append( [(wing_coord, gap_seq), None] )
                else:
                    s_size = (length-middle_size)//2
                    l_size = s_size + 1 

                    prime = []
                    #5<3
                    wing_coord = (s_size, middle_size, l_size)
                    gap_seq = sequence[s_size:-l_size]
                    prime.append( (wing_coord, gap_seq) )
                    
                    #5>3
                    wing_coord = (l_size, middle_size, s_size)
                    gap_seq = sequence[l_size:-s_size]
                    prime.append( (wing_coord, gap_seq) )
                    coordL.append( prime ) 
        return coordL
                    
    except Exception as e:
        print(traceback.format_exc())
        return e.argsss
        
    

In [70]:
_getWingCoord(result) # [((4, 10, 4), 'TATTTGTCGA'),((4, 10, 4), 'TATTTGTCGA')]

[[((4, 10, 4), 'TATTTGTCGA'), None],
 [((4, 10, 4), 'ATTTGTCGAG'), None],
 [((4, 10, 4), 'TTTGTCGAGT'), None],
 [((4, 10, 4), 'TTGTCGAGTC'), None],
 [((4, 10, 4), 'TGTCGAGTCT'), None],
 [((4, 10, 4), 'GTCGAGTCTT'), None],
 [((4, 10, 4), 'TCGAGTCTTA'), None],
 [((4, 10, 4), 'CGAGTCTTAT'), None],
 [((4, 10, 4), 'GAGTCTTATA'), None],
 [((4, 10, 4), 'AGTCTTATAT'), None],
 [((4, 10, 4), 'GTCTTATATG'), None],
 [((4, 10, 4), 'TCTTATATGA'), None],
 [((4, 10, 4), 'CTTATATGAG'), None],
 [((4, 10, 4), 'TTATATGAGA'), None],
 [((4, 10, 4), 'TATATGAGAA'), None],
 [((4, 10, 4), 'ATATGAGAAG'), None],
 [((4, 10, 4), 'TATGAGAAGA'), None],
 [((4, 10, 4), 'ATGAGAAGAC'), None],
 [((4, 10, 4), 'TGAGAAGACT'), None],
 [((4, 10, 4), 'GAGAAGACTT'), None],
 [((4, 10, 4), 'AGAAGACTTG'), None],
 [((4, 10, 4), 'GAAGACTTGG'), None],
 [((4, 10, 4), 'AAGACTTGGG'), None],
 [((4, 10, 4), 'AGACTTGGGT'), None],
 [((4, 10, 4), 'GACTTGGGTC'), None],
 [((4, 10, 4), 'ACTTGGGTCC'), None],
 [((4, 10, 4), 'CTTGGGTCCA'), None],
 

In [ ]:

def flaten_list(t):
    # [[a,b], [c,d]] -> [a,b,c,d]
    return [e for st in t for e in st]

def gapmer(result, middle_size=10, gapmer_coord='',target_assembly=["mm39"]):
    wingcoordL = _getWingCoord(result=result, middle_size=middle_size, gapmer_coord=gapmer_coord)

    # test conservation
    for assembly, coords in zip(target_assembly, wingcoordL):
        print(f"assembly: {assembly}")
        gapmer_filtered = []
        # test gap CpG
        gapL= [True for _item in [item[1].upper() for item in coords if item!=None] if "CG" in _item ]
        gap_condition = True if True in gapL else False
        wingL= [item[0] for item in coords if item!=None]

        for _coverage, _wobble in  zip(result[f"coverage_{assembly}"], result[f"wobble_{assembly}"]): 
            if gap_condition:
                gapmer_filtered.append(False)
                continue
            if str(_coverage) == "0"  or str(_coverage) == "None": # coverage값이 없을때만 가능 
                #print(_coverage, _wobble)# 
                if not _wobble or not _wobble["GU_humanC"]:
                    gapmer_filtered.append(False)
                    continue
                # GU_humanC에 값이 있다는 전제조건
                # no wobble in gap site
                #elif any([wing_coord[0]<=i<wing_coord[0]+wing_coord[1] for i in _wobble['GU_humanC']]): 
                elif any(
                    wing_coord[0] <= i < wing_coord[0] + wing_coord[1]
                    for i in _wobble['GU_humanC']
                    for wing_coord in wingL
                    ):
                    gapmer_filtered.append(False)
                    continue
                # GU_humanC외 다른 것을 합쳐서 중복을 제거하고 길이를 GU_humanC의 길이만큼 빼고 측정했을 때 0보다 크다면 contiune 
                # 그렇다면 두개가 동일한 길이면 0이니까 통과할 수 있음 
                # no additional wobble site
                elif len(set(flaten_list([v for k,v in _wobble.items() if k != 'GU_humanC'])) - set(_wobble['GU_humanC'])) > 0: 
                    gapmer_filtered.append(False)
                    continue
                else:
                    gapmer_filtered.append(True)
            else:
                gapmer_filtered.append(False)
                continue
        result[f"gapmer_filtered_{assembly}"] = gapmer_filtered
        result[f'gapmer_coords_{assembly}'] = [ ":".join(['_'.join(map(str, wing_coord)) for wing_coord in wingL])] * len(gapmer_filtered)
    return result
        #print(coords, len(coords))
    """
        # test gap CpG
        if "CG" in gaps.upper() :
            continue

        # test conservation
        # dowon add 
        for each_assembly in target_assembly:
            target_ass_cons = eval(cons_dict[each_assembly])
            
            if not target_ass_cons: # not conserved # 0이 아닌값이나 None이 아닌 값일시
                # dowon add
                if each_assembly in wob_dict.keys(): # if has wobble # wobble column이 없으면 continue
                    target_ass_dict = eval(wob_dict[each_assembly])
                    if not target_ass_dict or not target_ass_dict['GU_humanC']: # no A wobble or even no wobble
                        continue
                    if any([wingc[0]<=i<wingc[0]+wingc[1] for i in target_ass_dict['GU_humanC']]): # no wobble in gap site
                        continue
                    if len(set(flaten_list([v for k,v in target_ass_dict.items() if k != 'GU_humanC'])) - set(target_ass_dict['GU_humanC'])) > 0: # no additional wobble site
                        continue


                else:
                    continue
            restokL.append( (info, cons_dict, wob_dict, '_'.join(map(str,wingc))) )
        
    return restokL"""
    

In [127]:
#wingcoordL = _getWingCoord(result=result) 
#wingcoordL
gapmer_df = pd.DataFrame(gapmer(result=result, middle_size=10, gapmer_coord='', target_assembly=['mm39']))
gapmer_df

assembly: mm39


,Type,Gene,TranscriptID,Locus,Sequence,Length,RegionType,CommonSNP,Gquad,CpG,...,Homo_Dimer,Monomer,maf_seq_mm39,coverage_mm39,wobble_mm39,maf_seq_mm10,coverage_mm10,wobble_mm10,gapmer_filtered_mm39,gapmer_coords_mm39
0,e,TTN,NM_133379,chr2:178744408-178744425+,TTTATATTTGTCGAGTCT,18,"('TTN', 'NM_133379', 'antisense', [('utr3', (-...",False,False,1,...,-5.781484,-0.030474,TTTATATTTGTCGAGTCT:TTCATATTTATTATGTCT,5.0,False,None:None,None,"{'GU_humanC': [], 'GU_otherC': [], 'I_humanC':...",False,4_10_4
1,e,TTN,NM_133379,chr2:178744409-178744426+,TTATATTTGTCGAGTCTT,18,"('TTN', 'NM_133379', 'antisense', [('utr3', (0...",False,False,1,...,-5.785983,-0.028255,TTATATTTGTCGAGTCTT:TCATATTTATTATGTCTT,5.0,False,None:None,None,"{'GU_humanC': [], 'GU_otherC': [], 'I_humanC':...",False,4_10_4
2,e,TTN,NM_133379,chr2:178744410-178744427+,TATATTTGTCGAGTCTTA,18,"('TTN', 'NM_133379', 'antisense', [('utr3', (1...",False,False,1,...,-5.845405,-0.031674,TATATTTGTCGAGTCTTA:CATATTTATTATGTCTTA,5.0,False,None:None,None,"{'GU_humanC': [], 'GU_otherC': [], 'I_humanC':...",False,4_10_4
3,e,TTN,NM_133379,chr2:178744411-178744428+,ATATTTGTCGAGTCTTAT,18,"('TTN', 'NM_133379', 'antisense', [('utr3', (2...",False,False,1,...,-5.836573,-0.032538,ATATTTGTCGAGTCTTAT:ATATTTATTATGTCTTAT,4.0,False,None:None,None,"{'GU_humanC': [], 'GU_otherC': [], 'I_humanC':...",False,4_10_4
4,e,TTN,NM_133379,chr2:178744412-178744429+,TATTTGTCGAGTCTTATA,18,"('TTN', 'NM_133379', 'antisense', [('utr3', (3...",False,False,1,...,-5.825828,-0.034414,TATTTGTCGAGTCTTATA:TATTTATTATGTCTTATA,4.0,False,None:None,None,"{'GU_humanC': [], 'GU_otherC': [], 'I_humanC':...",False,4_10_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62993,e,TTN,NM_133379,chr2:178807401-178807418+,AGGCTGGGAATGCACGAC,18,"('TTN', 'NM_133379', 'antisense', [('utr5', (1...",False,False,1,...,-4.440665,-1.094383,AGGCTGGGAATGCACGAC:AGG-TGGGAATGCGCGAC,2.0,False,None:None,None,"{'GU_humanC': [], 'GU_otherC': [], 'I_humanC':...",False,4_10_4
62994,e,TTN,NM_133379,chr2:178807402-178807419+,GGCTGGGAATGCACGACT,18,"('TTN', 'NM_133379', 'antisense', [('utr5', (1...",False,False,1,...,-4.933975,-1.380091,GGCTGGGAATGCACGACT:GG-TGGGAATGCGCGACT,2.0,False,None:None,None,"{'GU_humanC': [], 'GU_otherC': [], 'I_humanC':...",False,4_10_4
62995,e,TTN,NM_133379,chr2:178807403-178807420+,GCTGGGAATGCACGACTG,18,"('TTN', 'NM_133379', 'antisense', [('utr5', (1...",False,False,1,...,-5.435167,-1.344265,GCTGGGAATGCACGACTG:G-TGGGAATGCGCGACTG,2.0,False,None:None,None,"{'GU_humanC': [], 'GU_otherC': [], 'I_humanC':...",False,4_10_4
62996,e,TTN,NM_133379,chr2:178807404-178807421+,CTGGGAATGCACGACTGC,18,"('TTN', 'NM_133379', 'antisense', [('utr5', (1...",False,False,1,...,-5.594740,-0.755325,CTGGGAATGCACGACTGC:-TGGGAATGCGCGACTGC,2.0,False,None:None,None,"{'GU_humanC': [], 'GU_otherC': [], 'I_humanC':...",False,4_10_4


In [128]:
gapmer_df[gapmer_df["gapmer_filtered_mm39"]==True]

,Type,Gene,TranscriptID,Locus,Sequence,Length,RegionType,CommonSNP,Gquad,CpG,...,Homo_Dimer,Monomer,maf_seq_mm39,coverage_mm39,wobble_mm39,maf_seq_mm10,coverage_mm10,wobble_mm10,gapmer_filtered_mm39,gapmer_coords_mm39


In [36]:
for _key in result.keys():
    if 'coverage' in _key or 'wobble' in _key:
        print(_key, result[_key])

coverage_mm39 [12, 12, 11, 12, 11, 11, 11, 11, 10, 9, 8, 7, 6, 5, 5, 10, 10, 9, 9, 11, 10, 10, 10, 10, 11, 11, 12, 13, 13, 13, 13, 13, 14, 15, 11, 11, 11, 12, 9, 9, 9, 9, 9, 12, 12, 11, 11, 11, 12, 13, 13, 13, 13, 13, 13, 13, 12, 12, 13, 14, 14, 14, 10, 10, 11, 11, 11, 10, 10, 10, 9, 8, 7, 7, 7, 7, 7, 6, 5, 4, 5, 5, 5, 4, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 5, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 6, 7, 7, 7, 8, 9, 10, 11, 12, 12, 13, 14, 14, 15, 14, 14, 14, 13, 14, 14, 13, 12, 11, 11, 11, 10, 10, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 8, 8, 8, 9, 9, 9, 10, 10, 11, 11, 12, 12, 12, 11, 12, 12, 11, 12, 12, 12, 12, 12, 11, 11, 11, 10, 10, 9, 9, 9, 8, 7, 8, 7, 8, 8, 9, 9, 9, 10, 10,

In [ ]:

def flaten_list(t):
    # [[a,b], [c,d]] -> [a,b,c,d]
    return [e for st in t for e in st]

def gapmer(result, middle_size=10, gapmer_coord='',target_assembly=["mm39"]):
    wingcoordL = _getWingCoord(result=result, middle_size=middle_size, gapmer_coord=gapmer_coord)
    for coords in zip(wingcoordL, ):
        # 두개이상을 감안하고 설계해야함
        for wing_coord, gap_seq in coords:
            # test gap CpG
            if "CG" in gap_seq.upper():
                continue
            # test conservation
            for assembly in target_assembly:
                for _coverage, _wobble in  zip(result[f"coverage_{assembly}"], result[f"wobble_{assembly}"]):
                    if _coverage
        print(coords, len(coords))
        """
        # test gap CpG
        if "CG" in gaps.upper() :
            continue

        # test conservation
        # dowon add 
        for each_assembly in target_assembly:
            target_ass_cons = eval(cons_dict[each_assembly])
            
            if not target_ass_cons: # not conserved
                # dowon add
                if each_assembly in wob_dict.keys(): # if has wobble
                    target_ass_dict = eval(wob_dict[each_assembly])
                    if not target_ass_dict or not target_ass_dict['GU_humanC']: # no A wobble or even no wobble
                        continue
                    if any([wingc[0]<=i<wingc[0]+wingc[1] for i in target_ass_dict['GU_humanC']]): # no wobble in gap site
                        continue
                    if len(set(flaten_list([v for k,v in target_ass_dict.items() if k != 'GU_humanC'])) - set(target_ass_dict['GU_humanC'])) > 0: # no additional wobble site
                        continue

                else:
                    continue

            restokL.append( (info, cons_dict, wob_dict, '_'.join(map(str,wingc))) )
        
    return restokL"""
    

AttributeError: 'tuple' object has no attribute 'upper'

In [ ]:
from collections import defaultdict

def split_list(foo, inL):
    left, right = [], []
    for i,e in enumerate(inL):
        if foo(e):
            left.append(e)
        else:
            right.append(e)
    return left, right

def flaten_list(t):
    # [[a,b], [c,d]] -> [a,b,c,d]
    return [e for st in t for e in st]

def gapmer_filter_line(info, cons_dict, wob_dict, target_assembly='mm39', gapmer_coord=''):
    #line 별로 값이 따로 들어
    seq = info[4]
    seq_len = len(seq)

    if gapmer_coord!='':
        wing_coord = tuple(map(int, gapmer_coord.split('_')))
        gap_seq = seq[wing_coord[0]:-wing_coord[2]]
        coordL = [(wing_coord, gap_seq)]

    else:
        coordL = []
        # sequence length가 2의 배수 
        if seq_len%2==0:
            # (20-10) //2 = 5 -> (5, 10, 5) seq[5:-5] result: (5,10,5),가운데 10개 seq 
            wing_size = (seq_len-10)//2
            wing_coord= (wing_size,10,wing_size)
            gap_seq   = seq[wing_size: -wing_size]
            coordL.append( (wing_coord, gap_seq) )

        else:
            s_size = (seq_len-10)//2
            l_size = s_size + 1 

            #5<3
            wing_coord = (s_size, 10, l_size)
            gap_seq = seq[s_size:-l_size]
            coordL.append( (wing_coord, gap_seq) )
            
            #5>3
            wing_coord = (l_size, 10, s_size)
            gap_seq = seq[l_size:-s_size]
            coordL.append( (wing_coord, gap_seq) )
    
    #result(wingc, gaps): (5,10,5), 가운데 10개 seq 
    restokL=[]
    for wingc, gaps in coordL:

        # test gap CpG
        if "CG" in gaps.upper() :
            continue

        # test conservation
        # dowon add 
        for each_assembly in target_assembly:
            target_ass_cons = eval(cons_dict[each_assembly])
            
            if not target_ass_cons: # not conserved
                # dowon add
                if each_assembly in wob_dict.keys(): # if has wobble
                    target_ass_dict = eval(wob_dict[each_assembly])
                    if not target_ass_dict or not target_ass_dict['GU_humanC']: # no A wobble or even no wobble
                        continue
                    if any([wingc[0]<=i<wingc[0]+wingc[1] for i in target_ass_dict['GU_humanC']]): # no wobble in gap site
                        continue
                    if len(set(flaten_list([v for k,v in target_ass_dict.items() if k != 'GU_humanC'])) - set(target_ass_dict['GU_humanC'])) > 0: # no additional wobble site
                        continue

                else:
                    continue

            restokL.append( (info, cons_dict, wob_dict, '_'.join(map(str,wingc))) )
        
    return restokL



def gapmer_filter_kmers(input_pathL, output_path,target_assembly='mm39'):
    # input_pathL 이전까지 만들어진 파일
    with open(output_path, 'w') as out:
        for path_num, input_path in enumerate(input_pathL):
            candf = open(input_path, 'r')
            # header filter
            header = candf.readline()
            header = header.rstrip().split('\t')

            # cons가 포함된 header 추출하고 나머지는 rest
            cons_colL, rest = split_list(lambda x: x.startswith('cons'), header)
            # wob가 포함된 header 추출하고 나머지는 info 
            wob_colL, info_colL = split_list(lambda x: x.startswith('wob'), rest)

            # header 에 번호를 매기고 tuple ex) [(0, header0), (1, header1)..] 
            info_colL_w_idx = [(i,v) for i,v in enumerate(header) if v in info_colL]
            cons_colL_w_idx = [(i,v.replace('cons_','')) for i,v in enumerate(header) if v in cons_colL]
            wob_colL_w_idx = [(i,v.replace('wob_','')) for i,v in enumerate(header) if v in wob_colL]
            
            #if len(wob_colL):
            #    wobble = True
            #else:
            #    wobble = False

            header = header + ['gapmer_coord']

            if path_num==0: #only use one header
                out.write('\t'.join(header)+'\n')

            for line in candf:
                tokenL = line.rstrip().split('\t')

                info = [tokenL[idx] for idx, colname in info_colL_w_idx] # header1에 매칭하는 아래의 값들을 가져옴
                cons_dict = {ass:tokenL[idx] for idx, ass in cons_colL_w_idx} # 위와 동일
                wob_dict  = {ass:tokenL[idx] for idx, ass in wob_colL_w_idx} # 위와 동일

                restokL = gapmer_filter_line(info, cons_dict, wob_dict, target_assembly=target_assembly, gapmer_coord='')

                for tok in restokL:
                    tok_line = tok[0] + [ tok[1][k] for _, k in cons_colL_w_idx ] + [ tok[2][k] for _, k in wob_colL_w_idx ] + [tok[3]]
                    #tok_line = tok[0] + [ tok[1][k] for _, k in cons_colL_w_idx ] + [tok[3]]
                    out.write('\t'.join(tok_line)+'\n')
        
            candf.close()

In [ ]:
# gapmer_filtered 옵션이 True라면, 추가 후처리
import os

gapmer_filtered =True
if gapmer_filtered:
    print()
    print('Gapmer_Filtering')
    output_path = os.path.join(
        os.path.dirname(os.path.realpath(__name__)),
        f'test_gapmer_filtered.tsv'
    )
    # 18mer 이상만 필터링 대상(예: k>=18)
    gapmer_filter_kmers(
        [kpath for kpath, k in iterRange if k >= 18],
        output_path,
        target_assembly=['mm39']#'mm39'
    )

In [96]:
for a in _getWingCoord(result):
    print(a)
    d= [True for item in [item[1] for item in a if item!=None] if "CG" in item ]
    test = True if True in d else False
    print(d,test)
    break

[((4, 10, 4), 'TATTTGTCGA'), None]
[True] True
